<a href="https://colab.research.google.com/github/wpfkcm33/korean_hate_speech/blob/main/bert_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [1]:
import torch

In [2]:
torch.cuda.empty_cache()

In [2]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [3]:
!pip install datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framewo

학습한 모델을 커뮤니티에서 공유하거나, 다른 pretrained model 처럼 쉽게 불러와 사용하고 싶다면
1. Hugging Face 웹사이트로부터 얻은 자신의 authentication token을 저장해야 합니다.(회원 가입이 필요하다면 [여기](https://huggingface.co/join)로)
2. 그 다음 아래 셀들의 주석을 풀어서 아이디와 비밀번호를 입력해주세요

In [4]:
from huggingface_hub import notebook_login

notebook_login()

그 다음 Git-LFS를 설치하고 Git을 설정해야 합니다. 아래 주석을 풀고 e-mail과 이름을 설정해주세요

In [5]:
!pip install hf-lfs
!git config --global user.email "endnjs33@gmail.com"
!git config --global user.name "DooWonLee"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 4.2 MB/s eta 0:00:00


In [6]:
import transformers
print(transformers.__version__)

4.37.2


In [7]:
import accelerate
print(accelerate.__version__)

0.27.2


In [8]:
# argment setting
task = "hate_speech"
model_checkpoint = "klue/roberta-base"
batch_size = 128

In [9]:
from datasets import load_dataset , DatasetDict
import pandas as pd

# 데이터셋 로드
dataset = load_dataset('Doowon96/hate_speech_labeled', split='train')

# 데이터셋을 7:3 비율로 분할
split_dataset = dataset.train_test_split(test_size=0.3)

# DatasetDict로 변환 (선택적)
dataset_dict = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

# DataFrame으로 변환
train_df = pd.DataFrame(dataset_dict['train'])
test_df = pd.DataFrame(dataset_dict['test'])

# 결측치 확인
print(train_df.isna().sum())
print(test_df.isna().sum())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

comments               0
contain_gender_bias    0
bias                   0
hate                   0
dtype: int64
comments               0
contain_gender_bias    0
bias                   0
hate                   0
dtype: int64


# 1. Loading the Dataset
[🤗 Datasets](https://github.com/huggingface/datasets) 라이브러리를 사용해 데이터셋을 다운로드 합니다. `load_dataset`을 이용하면 쉽게 다운로드 할 수 있습니다.

샘플 하나를 확인해보죠.

In [10]:
train_df.head(5)

,comments,contain_gender_bias,bias,hate
0,ㅋㅋㅋ 41살에20살이랑결혼한다데니 김새론영입하누 영계킬러ㅋㅋㅋ 사촌형 ㅇㅈㄹㅋㅋ후...,False,others,hate
1,자고로 여자는 25세전후 가슴C컵에 골반 넓고 하얗고 복스럽게 생겨야 똑똑하고 건강...,True,gender,hate
2,차예련씨는 여전히 곱고 이쁘네요~,False,none,none
3,"문재인은 조작사기단 sm과 방송고위간부및 제작진,언론과의 유착비리 조사하고 구속해라...",False,others,offensive
4,화교년 아니냐,False,others,hate


In [11]:
test_df.head(5)

,comments,contain_gender_bias,bias,hate
0,얘는 먼데 방송한다고 난리??? 고쳐도 서세원이야,False,none,hate
1,아직도 티비속 화면으로 성격판단하는 멍충한분들이 많네..ㅋㅋ,False,none,offensive
2,승리는 일단 마약을 안했자나 ㄹㅇ,False,none,none
3,돼지는 영원한 돼지다,False,others,hate
4,스페인 광장에서 cf 찍을때 드라마 하버드 찍을때 미모가 절정이었다. 그러더니 점점...,False,none,hate


데이터셋이 어떻게 생겼는지 한 눈에 확인해봅시다. 아래함수는 데이터셋에서 임의의 샘플을 추출해 확인합니다.

# 2. Processing the data

🤗 Transformers `Tokenizer`는 모델의 입력 텍스트를 토크나이징하고(또한 각 토큰들에 할당된 IDs로 변환하는 작업도 함께 수행), 모델이 입력받는 포맷으로 만들어 줍니다(각 모델이 입력 받아야할 입력 포맷, BERT Pretraining의 경우 attention mask, token type ids 까지 함꼐 처리).

이 모든 작업을 하기 위해서 먼저 `AutoTokenizer.from_pretrained`를 사용해 토크나이저를 준비해야합니다. 이걸 사용하면

- 사용하고자 하는 모델과 관련된 토크나이저를 얻을 수 있습니다.
- 특정 체크포인트를 pretraining할 때 사용된 vocabulary를 다운로드 할 수 있습니다.

vocabulary는 캐시되므로, 다음번에 셀을 실행할 때 다시 다운로드 되지 않습니다.

In [12]:
model_checkpoint

'klue/roberta-base'

In [13]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

토크나이저를 이용해 문장을 인코딩할 수 있습니다. 하나의 문장을 입력하거나 문장 쌍을 입력할 수 있습니다

In [14]:
tokenizer("가짜연구소 3기 KLUE 벤치마크 팀 화이팅!!!", "잘 부탁 드려요")

{'input_ids': [0, 8711, 19648, 23, 2015, 47, 2237, 2309, 2105, 9262, 9112, 1823, 21121, 5, 5, 5, 2, 1521, 5527, 20469, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

YNAT 태스크에서는 분류 대상 문장이 포함된 열인 'title'을 모델 입력으로 사용합니다.

전처리를 위해 사용할 함수를 작성해봅시다. `tokenizer`의 입력으로 dataset의 'text'열을 입력하면서 `truncation=True`를 설정하면됩니다.
 이 옵션은 모델이 입력 받을 수 있는 토큰 최대 길이를 벗어난 토큰들을 잘라주는 옵션입니다.

In [15]:
from datasets import Dataset

# Pandas DataFrame을 Hugging Face의 Dataset으로 변환
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

# 카테고리를 정수로 매핑하는 딕셔너리
category_to_id = {'none': 0, 'offensive': 1, 'hate': 2}

def preprocess_function(examples):
    # 제목을 토크나이즈
    tokenized_titles = tokenizer(examples['comments'], truncation=True, padding=True)

    # 카테고리 레이블을 정수로 변환
    tokenized_titles['labels'] = [category_to_id[category] for category in examples['hate']]

    return tokenized_titles

# 전처리 함수 적용
encoded_dataset_train = dataset_train.map(preprocess_function, batched=True)
encoded_dataset_test = dataset_test.map(preprocess_function, batched=True)


Map:   0%|          | 0/5527 [00:00<?, ? examples/s]

Map:   0%|          | 0/2369 [00:00<?, ? examples/s]

이 함수는 입력 데이터를 하나 이상 받을 수 있어 배치로 처리 가능합니다.
이 경우 각각의 key의 value는 리스트 형태로 반환됩니다.

이 함수를 전체 데이터셋의 문장에 적용시키기 위해서, 앞서 생성한 `dataset` 객체의 `map` 메서드를 사용합니다. 이 때 `dataset`의 모든 splits에 적용되어 한 번의 실행 만으로 'train', 'validation' 셋이 모두 모델의 입력으로 변환됩니다.

변환 결과는 🤗 Datasets 라이브러리에 의해 자동으로 캐시되어 다음 번에 노트북을 실행할 때는 캐시된 데이터를 불러와 시간을 절약합니다. 🤗 Datasets 라이브러리는 `map` 메서드에 입력되는 함수의 수정 여부를 체크하여 캐시된 데이터의 사용여부를 결정합니다. 이때 🤗 Datasets 캐시된 파일을 사용여부를 알려주며, `map` 메서드 호출 시 `load_from_cache_file=False`으로 변환을 강제할 수 있습니다.
  
`batched=True`를 사용해서 배치로 묶어서 텍스트를 인코딩 한 것을 볼 수 있는데, 이건 앞서 로드한 fast tokenizer의 이점을 활용하기 위함입니다. fast tokenizer는 멀티 쓰레딩을 사용하여 텍스트를 배치 형태로 거의 동시에 처리 합니다.

# 3. Fine-tuning the model

데이터가 준비되었으니, Pretrained model을 다운 받아 fine-tune 해봅시다. YNAT은 문장 분류와 관련된 task이므로,  `AutoModelForSequenceClassification` 클래스를 사용합니다. tokenizer 때처럼,  `from_pretrained` 메서드로 모델을 다운로드 받고 캐시합니다.  
딱 한 가지 설정해줘야할 것은 label의 개수입니다(YNAT은 총 7개를 label로 구성됨):

In [16]:
encoded_dataset_train.features['labels']

Value(dtype='int64', id=None)

이 노트북은 [Model Hub](https://huggingface.co/models)에서 Pretraining된 모델이 존재할 경우(`[모델명]ForPreTraining`) 모델 체크포인트를 다운로드하여 YNAT task를 학습합니다.

In [17]:
model_checkpoint

'klue/roberta-base'

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


위의 경고메시지는 다음을 의미합니다.

- `BertForPreTraining`엔 있지만 `BertForSequenceClassification`엔 없는 레이어는 버리고
- `BertForSequenceClassification` 엔 있지만 `BertForPreTraining`엔 없는 레이어는 랜덤 초기화.

따라서 `BertForSequenceClassification` 모델을 fine-tune 하지 않으면 좋은 성능을 얻지 못할 것이라는 것이죠🙂[(reference)](https://github.com/huggingface/transformers/issues/5421)

HuggingFace 라이브러리에서는 `Trainer` 객체를 사용하여 복잡한 학습 코드를 짤 필요 없이 편리하게 학습할 수 있습니다.  
`Trainer` 객체는 `TrainingArgument`를 입력 받는데 모델 학습을 위해 설정하는 값들이 attribute로 들어있는 클래스 입니다. 입력받는 인자 중에 필수적인 것은 `output_dir`인데, 모델의 checkpoint를 저장하기 위한 경로를 의미합니다.

In [19]:
from transformers import TrainingArguments
import os

model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "ynat")
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch",  # 변경: 에폭 단위로 모델 저장
    load_best_model_at_end=True,
    # save_steps 설정 제거 또는 주석 처리
    # save_steps = 500,  # 주석 처리

    # Dataset
    num_train_epochs=5,  # 훈련 에폭 수
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    # Optimizer
    learning_rate=2e-5,  # 5e-5
    weight_decay=0.01,  # 0
    warmup_steps=10,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,

    # Evaluation
    metric_for_best_model='eval_f1',
    evaluation_strategy = "epoch",  # 변경: 에폭 단위로 평가

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,  # 로그 저장 디렉토리 설정 유지
    report_to='wandb',  # Weights & Biases 로깅 설정 유지

    # Randomness
    seed=42,  # 랜덤 시드 설정 유지
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1760: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Doowon96/roberta-base-finetuned-hate_speech).
  warnings.warn(


학습을 끝까지 수행한 모델이 최고 성능을 내는 모델은 아닙니다. 따라서 학습이 종료되었을 때 `Trainer` 객체가 `metric_name`에서 설정한 `metric`에 따라 가장 좋은 성능을 보인 모델을 불러오도록 `metric_for_best_model` 파라미터를 설정합니다.  
주석처리한 마지막 두 파라미터는 학습이 끝날 때 모델을 [Hub](https://huggingface.co/models)에 push 할 때 설정합니다.

마지막으로 `Trainer` 세팅을 위해서 필요한 것은 metric을 계산하는 방법을 정의해줍니다. 먼저 `datasets` 라이브러리에서 F1-score metric을 로드하여 `meric_macrof1` 변수에 할당한 뒤, 모델의 출력값을 입력으로 받는 `compute_metrics` 함수를 정의하여 `meric_macrof1`을 계산한 결과를 출력합니다. 입력에 적절하도록 모델의 출력값을 전처리 해주는 과정이 포함됩니다.

In [20]:
# datasets 라이브러리에서 제공하는 Evaluation metric의 리스트를 확인합니다.
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

<ipython-input-20-e741eeb3c60a>:3: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


accuracy, bertscore, bleu, bleurt, brier_score, cer, character, charcut_mt, chrf, code_eval, comet, competition_math, confusion_matrix, coval, cuad, exact_match, f1, frugalscore, glue, google_bleu, indic_glue, mae, mahalanobis, mape, mase, matthews_correlation, mauve, mean_iou, meteor, mse, nist_mt, pearsonr, perplexity, poseval, precision, r_squared, recall, rl_reliability, roc_auc, rouge, sacrebleu, sari, seqeval, smape, spearmanr, squad, squad_v2, super_glue, ter, trec_eval, wer, wiki_split, xnli, xtreme_s, AlhitawiMohammed22/CER_Hu-Evaluation-Metrics, BucketHeadP65/confusion_matrix, BucketHeadP65/roc_curve, CZLC/rouge_raw, DaliaCaRo/accents_unplugged_eval, DarrenChensformer/eval_keyphrase, DarrenChensformer/relation_extraction, DoctorSlimm/bangalore_score, DoctorSlimm/kaushiks_criteria, Drunper/metrica_tesi, Felipehonorato/eer, Fritz02/execution_accuracy, GMFTBY/dailydialog_evaluate, GMFTBY/dailydialogevaluate, He-Xingwei/sari_metric, Ikala-allen/relation_extraction, JP-SystemsX/nD

In [21]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [22]:
import evaluate

# 평가 지표 리스트를 로드합니다.
metrics_list = evaluate.list_evaluation_modules()

# 리스트의 길이와 함께 평가 지표를 출력합니다.
print(len(metrics_list))
print(', '.join(metric for metric in metrics_list))


146
lvwerra/test, jordyvl/ece, angelina-wang/directional_bias_amplification, cpllab/syntaxgym, lvwerra/bary_score, hack/test_metric, yzha/ctc_eval, codeparrot/apps_metric, mfumanelli/geometric_mean, daiyizheng/valid, erntkn/dice_coefficient, mgfrantz/roc_auc_macro, Vlasta/pr_auc, gorkaartola/metric_for_tp_fp_samples, idsedykh/metric, idsedykh/codebleu2, idsedykh/codebleu, idsedykh/megaglue, cakiki/ndcg, Vertaix/vendiscore, GMFTBY/dailydialogevaluate, GMFTBY/dailydialog_evaluate, jzm-mailchimp/joshs_second_test_metric, ola13/precision_at_k, yulong-me/yl_metric, abidlabs/mean_iou, abidlabs/mean_iou2, KevinSpaghetti/accuracyk, NimaBoscarino/weat, ronaldahmed/nwentfaithfulness, Viona/infolm, kyokote/my_metric2, kashif/mape, Ochiroo/rouge_mn, giulio98/code_eval_outputs, leslyarun/fbeta_score, giulio98/codebleu, anz2/iliauniiccocrevaluation, zbeloki/m2, xu1998hz/sescore, dvitel/codebleu, NCSOFT/harim_plus, JP-SystemsX/nDCG, sportlosos/sescore, Drunper/metrica_tesi, jpxkqx/peak_signal_to_nois

In [23]:
metric_macrof1 = evaluate.load('f1')

In [24]:
def compute_metrics(eval_pred):
    predictions = eval_pred.predictions.argmax(-1)
    labels = eval_pred.label_ids
    return metric_macrof1.compute(predictions=predictions,
                                  references=labels, average='macro')

In [25]:
encoded_dataset_train.info

DatasetInfo(description='', citation='', homepage='', license='', features={'comments': Value(dtype='string', id=None), 'contain_gender_bias': Value(dtype='bool', id=None), 'bias': Value(dtype='string', id=None), 'hate': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Value(dtype='int64', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, dataset_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

In [26]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


앞에서 `tokenizer`를 사용해 전처리를 했음에도 `Trainer`의 입력으로 다시 넣는 이유는 패딩을 적용해서 입력 샘플들을 동일한 길이로 만들기 위해 (데이터 로더의 마지막 과정에서) 사용하기 때문입니다. 모델에 따라 패딩에 대한 기본 설정이 다르기 때문에(왼쪽 패딩, 오른쪽 패딩, 또는 패딩 인덱스 번호 설정 등) `Trainer`는 이와 관련된 작업을 수행할 수 있은 `tokenizer` 객체를 사용합니다. `tokenizer` 대신에 `data_collator`를 커스텀하여 `tokenizer`를 적용하였을 때와 동일한 형태의 입력을 만들어 줄 수도 있습니다.

huggingface는 모델 학습 로그를 기록할때 Tensorboard 또는 [Weights & Biases](https://wandb.ai/site)를 사용할 수 있습니다. 여기서는 Weights & Biases를 사용해 보겠습니다.

In [27]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

실험 관리를 위해서 id값을 생성합니다.

In [28]:
id = wandb.util.generate_id()
print(id)

gkottemi


- project : 실험기록을 관리할 프로젝트 이름. 없을 시 입력받은 이름으로 생성, 여기선 예시로 klue로 설정
- entity : weights & biases 사용자명 또는 팀 이름
- id : 실험에 부여된 고유 아이디
- name : 실험에 부여한 이름
- resume : 실험을 재개할 떄, 실험에 부여한 고유 아이디를 입력

In [29]:
wandb.init(project='hate_speech',
           entity='wpfkcm33',
           id='gkottemi',
        #    name='[EXPERIMENT_NAME]',
           resume=True,
            )

wandb: Currently logged in as: wpfkcm33. Use `wandb login --relogin` to force relogin


In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.828418,0.589995
2,No log,0.791077,0.609088
3,No log,0.795406,0.623139
4,No log,0.814190,0.635730
5,No log,0.852321,0.622723


TrainOutput(global_step=220, training_loss=0.6801412409002131, metrics={'train_runtime': 423.2409, 'train_samples_per_second': 65.294, 'train_steps_per_second': 0.52, 'total_flos': 1164518403341580.0, 'train_loss': 0.6801412409002131, 'epoch': 5.0})

`evaluate` 메서드를 사용하여 `Trainer`가 best 모델로 불러온 모델의 성능을 확인해볼 수 있습니다.

In [31]:
trainer.evaluate()

{'eval_loss': 0.8141902089118958,
 'eval_f1': 0.6357304588270033,
 'eval_runtime': 9.2493,
 'eval_samples_per_second': 256.128,
 'eval_steps_per_second': 2.054,
 'epoch': 5.0}

학습이 끝나면 wandb 를 꺼줍니다

In [32]:
wandb.finish()

eval/f1,▁▄▆█▆█
eval/loss,▅▁▁▄█▄
eval/runtime,██▇██▁
eval/samples_per_second,▁▁▂▁▁█
eval/steps_per_second,▁▁▂▁▁█
train/epoch,▁▃▅▆███
train/global_step,▁▃▅▆███
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


현재 tokenizer를 비롯한 모델을 Hub에 업로드하려면 `push_to_hub()` 메서드를 사용합니다.

In [33]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Doowon96/roberta-base-finetuned-hate_speech/commit/4fd12c48d09ac6c899cd065d23683f96513bcc35', commit_message='End of training', commit_description='', oid='4fd12c48d09ac6c899cd065d23683f96513bcc35', pr_url=None, pr_revision=None, pr_num=None)

업로드한 모델을 `Hub-user-name/사용자가 지정한 이름`으로 바로 다운로드하여 사용할 수 있습니다.

In [34]:
from transformers import AutoModelForSequenceClassification
num_labels = 3
# {HuggingFace Model Hub 사용자 아이디}/{push_to_hub_model_id에서 설정한 값}
model = AutoModelForSequenceClassification.from_pretrained('Doowon96/roberta-base-finetuned-hate_speech', num_labels=num_labels)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

# 4. Hyperparameter search

`Trainer`는 [optuna](https://optuna.org/) 또는 [Ray Tune](https://docs.ray.io/en/latest/tune/)를 이용한 hyperparameter search를 지원합니다. 이 섹션을 실습하기 위해서 두 라이브러리 모두 설치되어 있어야 합니다. 아래 주석을 풀고 실행시켜 라이브러리를 설치한 뒤 다음 셀을 실행시켜 주세요

In [35]:
! pip install optuna
! pip install ray[tune]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.4 MB/s eta 0:00:00


In [36]:
from transformers import TrainingArguments
import os

model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "ynat")
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch",  # 변경: 에폭 단위로 모델 저장
    load_best_model_at_end=True,
    # save_steps 설정 제거 또는 주석 처리
    # save_steps = 500,  # 주석 처리

    # Dataset
    num_train_epochs=5,  # 훈련 에폭 수
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    # Optimizer
    learning_rate=2e-5,  # 5e-5
    weight_decay=0.01,  # 0
    warmup_steps=10,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,

    # Evaluation
    metric_for_best_model='eval_f1',
    evaluation_strategy = "epoch",  # 변경: 에폭 단위로 평가

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,  # 로그 저장 디렉토리 설정 유지
    report_to='wandb',  # Weights & Biases 로깅 설정 유지

    # Randomness
    seed=42,  # 랜덤 시드 설정 유지
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1760: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Doowon96/roberta-base-finetuned-hate_speech).
  warnings.warn(


hyperparameter search 동안에 `Trainer`는 학습을 여러번 실행합니다. 따라서 모델이 매 학습마다 다시 초기화 될 수 있도록 모델이 함수에 의해 정의되도록 합니다.

In [47]:
model_checkpoint


'klue/roberta-base'

In [37]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

그 다음, 모델을 직접 입력하는 대신 `model_init`으로 초기화하는 함수를 입력합니다. 나머지는 앞에서 했던 것과 거의 동일하게 `Trainer`를 초기화 해주었습니다.

전체 데이터셋에 대해서 사용할 떄 몇몇 태스크에서는 시간이 오래 걸릴 수 있습니다. 이럴 땐 아래 예시 코드처럼  `dataset` 객체의 `shard` 메서드를 이용해 데이터 셋의 일부(예시에선 1/10)만을 선택하여 hyperparameter를 찾아보는 것도 한 방법이 될 수 있습니다. 찾은 hyperparameter를 전체 데이터셋에 대해 학습할 때 적용하여 최종 모델을 학습 시킵니다.

```python
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10)
```


In [46]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
import wandb

In [41]:
wandb.login()

True

In [42]:
id = wandb.util.generate_id()
print(id)

s7vhtr42


- project : 실험기록을 관리할 프로젝트 이름. 없을 시 입력받은 이름으로 생성
- entity : weights & biases 사용자명 또는 팀 이름
- id : 실험에 부여된 고유 아이디
- name : 실험에 부여한 이름
- resume : 실험을 재개할 떄, 실험에 부여한 고유 아이디를 입력

In [43]:
wandb.init(project='roberta_para1',
           entity='wpfkcm33',
           id='[s7vhtr42]',
        #    name='[EXPERIMENT_NAME]'
            resume=True,
           )

`Trinaer` 객체의 `hyperparameter_search` 메서드를 사용해 hyperparameter search를 수행합니다. 입력 파라미터는 다음과 같습니다.
- `hp_space` : hyperparameter search를 수행할 딕셔너리를 반환하는 함수를 입력받습니다. 값을 설정하지 않을 경우 optuna의 기본값을 사용합니다.
 - optuna를 사용할 경우:  
```python
def my_hp_space(trial):
        return {
            "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
            "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
            "seed": trial.suggest_int("seed", 1, 40),
            "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
        }
```
 - ray를 사용할 경우:  
```python
def my_hp_space_ray(trial):
        from ray import tune

        return {
            "learning_rate": tune.loguniform(1e-4, 1e-2),
            "num_train_epochs": tune.choice(range(1, 6)),
            "seed": tune.choice(range(1, 41)),
            "per_device_train_batch_size": tune.choice([4, 8, 16, 32, 64]),
        }
```


- `computive_objective` : 최대화하거나 최소화할 목적함수를 받습니다. 기본값으로 모델의 `evaluate` 메서드에 의해 반환되는 metric값(여기선 F1-score)를 사용합니다.
    ```python
    def my_objective(metrics):
        return metrics["eval_f1"]
    ```
- `n_trials` : 테스트할 실험의 개수를 설정합니다(기본값 100).
- `direction` : `computive_objective`값의 최적화의 방향을 정합니다. `'minimize'`(기본값) 또는 `'maximize'`.

In [48]:
#오래걸림! 사용전 logging옵션+콜랩 유지시간+n_trials횟수 확인
best_run = trainer.hyperparameter_search(n_trials=5, direction="maximize")

[I 2024-02-19 05:43:01,894] A new study created in memory with name: no-name-c9e694d9-e61f-4720-92b3-2c69fcd89cd2
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.882500,0.828630,0.592249
2,0.760600,0.838506,0.596660


[I 2024-02-19 05:49:14,031] Trial 0 finished with value: 0.5966597659617285 and parameters: {'learning_rate': 1.8850033551830082e-06, 'num_train_epochs': 2, 'seed': 19, 'per_device_train_batch_size': 4}. Best is trial 0 with value: 0.5966597659617285.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/f1,▁█
eval/loss,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▃▄▄▆▇██
train/global_step,▁▃▄▄▆▇██
train/learning_rate,█▆▅▃▁
train/loss,█▄▃▁▁
train/total_flos,▁
train/train_loss,▁


Epoch,Training Loss,Validation Loss,F1
1,No log,0.817370,0.602988
2,No log,0.826029,0.620901
3,No log,0.955649,0.634850
4,No log,1.142534,0.637215
5,No log,1.375547,0.631423


...

Epoch,Training Loss,Validation Loss,F1
1,0.893100,0.793933,0.619872
2,0.703400,0.818286,0.631251
3,0.407800,1.034998,0.634705


[I 2024-02-19 06:13:14,435] Trial 3 finished with value: 0.634705330626114 and parameters: {'learning_rate': 2.237588569580688e-05, 'num_train_epochs': 3, 'seed': 39, 'per_device_train_batch_size': 8}. Best is trial 3 with value: 0.634705330626114.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/f1,▁▆█
eval/loss,▁▂█
eval/runtime,▆█▁
eval/samples_per_second,▃▁█
eval/steps_per_second,▃▁█
train/epoch,▁▂▃▅▅███
train/global_step,▁▂▃▅▅███
train/learning_rate,█▆▃▁
train/loss,█▅▃▁
train/total_flos,▁
train/train_loss,▁


Epoch,Training Loss,Validation Loss,F1
1,No log,0.805084,0.634905
2,0.798300,0.820810,0.635980
3,0.512200,0.918371,0.635837
4,0.512200,1.119126,0.634233


[I 2024-02-19 06:20:40,682] Trial 4 finished with value: 0.6342325467636821 and parameters: {'learning_rate': 2.402778145515359e-05, 'num_train_epochs': 4, 'seed': 12, 'per_device_train_batch_size': 16}. Best is trial 3 with value: 0.634705330626114.


`hyperparameter_search` 메서드는 `BestRun` 객체를 반환합니다. 이 객체는 최대화된 objective의 값과 이때 선택된 hyperparameter를 포함합니다.

In [49]:
best_run

BestRun(run_id='3', objective=0.634705330626114, hyperparameters={'learning_rate': 2.237588569580688e-05, 'num_train_epochs': 3, 'seed': 39, 'per_device_train_batch_size': 8}, run_summary=None)

In [59]:
from transformers import TrainingArguments

# best_run의 결과에서 최적의 하이퍼파라미터 추출
best_hyperparameters = {
    "learning_rate": 2.237588569580688e-05,
    "num_train_epochs": 3,
    "seed": 39,
    "per_device_train_batch_size": 8
}

# 최적의 하이퍼파라미터로 TrainingArguments 설정
training_args = TrainingArguments(
    output_dir=output_dir,  # 모델과 체크포인트를 저장할 디렉토리
    num_train_epochs=best_hyperparameters["num_train_epochs"],  # 최적의 훈련 에폭 수
    learning_rate=best_hyperparameters["learning_rate"],  # 최적의 학습률
    per_device_train_batch_size=best_hyperparameters["per_device_train_batch_size"],  # 최적의 배치 사이즈
    evaluation_strategy="epoch",  # 에폭마다 평가
    save_strategy="epoch",  # 에폭마다 모델 저장
    load_best_model_at_end=True,  # 훈련 종료 시 최고 성능 모델 로드
    metric_for_best_model="eval_f1",  # 최고 모델 선정 기준
    push_to_hub=True,  # Hugging Face Hub에 모델 업로드
    push_to_hub_model_id=f"{model_name}-finetuned-{task}-best",  # 업로드할 모델 ID
    logging_dir=logging_dir,  # 로깅 디렉토리
    seed=best_hyperparameters["seed"],  # 최적의 시드 값
)

# 위 설정을 사용하여 Trainer 인스턴스 생성 및 모델 훈련 코드를 이어서 작성



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1760: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Doowon96/roberta-base-finetuned-hate_speech-best).
  warnings.warn(


In [60]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.903000,0.795680,0.602813
2,0.723600,0.796454,0.631579
3,0.462100,0.999242,0.629015


TrainOutput(global_step=2073, training_loss=0.6663947365333421, metrics={'train_runtime': 354.4959, 'train_samples_per_second': 46.773, 'train_steps_per_second': 5.848, 'total_flos': 696762358254900.0, 'train_loss': 0.6663947365333421, 'epoch': 3.0})

In [61]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1708325183.2c8afbeedd1a.444.1:   0%|          | 0.00/6.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Doowon96/roberta-base-finetuned-hate_speech-best/commit/65654632707aba0cb4cb6611dcef5b0ad5273c5f', commit_message='End of training', commit_description='', oid='65654632707aba0cb4cb6611dcef5b0ad5273c5f', pr_url=None, pr_revision=None, pr_num=None)

# Appendix

## 1. optuna를 사용한 hyperparameter search code snippet

```python
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

class MemorySaverCallback(TrainerCallback):
    "A callback that deleted the folder in which checkpoints are saved, to save memory"
    def __init__(self, run_name):
        super(MemorySaverCallback, self).__init__()
        self.run_name = run_name

    def on_train_begin(self, args, state, control, **kwargs):
        print("Removing dirs...")
        if os.path.isdir(f'./{self.run_name}'):
            import shutil
            shutil.rmtree(f'./{self.run_name}')
        else:
            print("\n\nDirectory does not exists")

training_args = TrainingArguments(
    RUN_NAME,
    num_train_epochs=15,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=1,
    logging_first_step=False,
    overwrite_output_dir=True,
    save_strategy="no",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
)

trainer = Trainer(
    model_init=partial(MyNet,2),
    args=training_args,
    train_dataset=training_opos.select(range(2000)),
    eval_dataset=validating_opos,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2), MemorySaverCallback(RUN_NAME)]
)

def my_hp_space_optuna(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-6, 2e-4, log=True),
        "warmup_steps":  trial.suggest_float("warmup_steps", 0., 0.9, step=0.3),
        "weight_decay":  trial.suggest_float("weight_decay", 1e-6, 1e-1)
    }
def my_objective(metrics):
    return metrics["eval_f1"]

sa = trainer.hyperparameter_search(
    direction="maximize",
    n_trials=1,
    hp_space=my_hp_space_optuna,
    compute_objective=my_objective
)
```

# Reference
- [Text Classification on GLUE](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb#scrollTo=71pt6N0eIrJo)
- [HuggingFace Datasets Docs](https://huggingface.co/docs/datasets/index.html)
- [HuggingFace Transformers Docs](https://huggingface.co/transformers/index.html)
- [Using hyperparameter-search in Trainer](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/55)